In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from concurrent.futures import ThreadPoolExecutor
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [ ]:
# Diretório base onde os arquivos CSV estão localizados
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho_base = diretório_princapl / 'Bases'
caminho_arquivos = diretório_princapl / 'data'

# Definir caminho dos arquivos 
arquivos = {
    'Bitcoin': caminho_base / 'Bitcoin Historical Data.csv',
    'Ethereum': caminho_base / 'Ethereum Historical Data.csv',
    'BNB': caminho_base / 'BNB Historical Data.csv',
    'Solana': caminho_base / 'Solana Historical Data.csv',
}

# Ler o arquivo ano.txt e pegar o ano
with open(diretório_princapl/'data'/'ano.txt', 'r') as file:
    ano = int(file.read().strip())

# Função para tratar os dados de cada criptomoeda
def tratar_dados_cripto(caminho_csv):
    # Ler o arquivo CSV
    acao = pd.read_csv(caminho_csv)
    acao['Date'] = pd.to_datetime(acao['Date'])

    return acao

# Aplicar a função para cada criptomoeda e armazenar o resultado em um DataFrame
df_bitcoin = tratar_dados_cripto(arquivos['Bitcoin'])
df_ethereum = tratar_dados_cripto(arquivos['Ethereum'])
df_bnb = tratar_dados_cripto(arquivos['BNB'])
df_solana = tratar_dados_cripto(arquivos['Solana'])


def previsao(acao, neurons, batch_size, epochs, nome):
    # Separar um df no qual vai conter apenas os dado para treinamento
    # Que é com base no ano escolhido pelo jogador
    df_treinamento = acao[acao['Date'].dt.year < ano]

    # Apenas cotação dos dados de treinamento 
    cotacao_treinamento = df_treinamento['Price'].to_numpy().reshape(-1, 1)

    # Cotação dos dados da base toda
    cotacao = acao['Price'].to_numpy().reshape(-1, 1)

    # Armazenar tamanho dos dados de treinamento
    tamanho_dados_treinamento = int(len(cotacao_treinamento) * 1)

    #escalar os dados entre 0 e 1, para deixar mais fácil o processamento
    #dados em escala pré definidas são mais fáceis de lidar. 
    escalador = MinMaxScaler(feature_range=(0, 1))

    dados_entre_0_e_1_treinamento = escalador.fit_transform(cotacao[0: tamanho_dados_treinamento, :])
    dados_entre_0_e_1_restantes = escalador.transform(cotacao[tamanho_dados_treinamento: , :])

    dados_entre_0_e_1 = list(dados_entre_0_e_1_treinamento.reshape(
        len(dados_entre_0_e_1_treinamento))) + list(dados_entre_0_e_1_restantes.reshape(len(dados_entre_0_e_1_restantes)))
                                                    
    dados_entre_0_e_1 = np.array(dados_entre_0_e_1).reshape(len(dados_entre_0_e_1), 1)
    dados_para_treinamento = dados_entre_0_e_1[0: tamanho_dados_treinamento, :]

    #dados que serão usados para gerar o resultado
    treinamento_x = []
    #cotação que aconteceu de fato
    treinamento_y = []

    for i in range(60, len(dados_para_treinamento)):

        #60 ultimos dias
        treinamento_x.append(dados_para_treinamento[i - 60: i, 0])
        #cotacao
        treinamento_y.append(dados_para_treinamento[i, 0])

        if i <= 61:
            print(treinamento_x)
            print(treinamento_y)
    
    #transformando as listas em arrays e dando reshape 3d 

    treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
    treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

    #construindo o modelo
    modelo = Sequential()

    modelo.add(LSTM(neurons, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
    modelo.add(LSTM(neurons//2, return_sequences=False))
    modelo.add(Dense((neurons//2)//2))
    modelo.add(Dense(1))

    treinamento_x.shape[1]

    #copilando o modelo
    modelo.compile(optimizer="adam", loss="mean_squared_error") 

    modelo.fit(treinamento_x, treinamento_y, batch_size=batch_size, epochs=epochs)

    dados_teste = dados_entre_0_e_1[tamanho_dados_treinamento - 60:, :]

    predicao_x = []
    predicao_y = cotacao[tamanho_dados_treinamento: , :] 

    for i in range(60, len(dados_teste)):
        predicao_x.append(dados_teste[i - 60: i, 0])

    # Reshape
    predicao_x = np.array(predicao_x)
    predicao_x = predicao_x.reshape(predicao_x.shape[0], predicao_x.shape[1], 1)

    # Pegando predições do modelo
    predicoes = modelo.predict(predicao_x)

    # Tirando a escala dos dados
    predicoes = escalador.inverse_transform(predicoes)

    # dados do modelo
    treinamento = acao.iloc[:tamanho_dados_treinamento, :]
    df_previsao = pd.DataFrame({"Date": acao['Date'].iloc[tamanho_dados_treinamento:],
                            "predicoes": predicoes.reshape(len(predicoes))})
    
    # Setar a data como index dos df (df_previsao e treinamento)
    df_previsao.set_index('Date', inplace=True)
    treinamento.set_index('Date', inplace=True)

    df_previsao.sort_index()

    df_previsao_semanal = df_previsao
    df_previsao_semanal['Semana'] = ((df_previsao_semanal.index - df_previsao_semanal.index.min()).days // 7) + 1

    df_previsao_semanal = df_previsao_semanal.reset_index() 


    df_previsao_semanal.reset_index() 
    # Agrupar por semana e calcular o preço médio semanal das previsões
    df_teste_semana = df_previsao_semanal.groupby('Semana').agg({
        'predicoes': 'mean',   # Previsão média por semana
        'Date': 'first'        # Pega a primeira data da semana 
    })

    # Adicionar colunas de mês e ano com base na coluna 'Data'
    df_teste_semana['Mes'] = df_teste_semana['Date'].dt.month
    df_teste_semana['Ano'] = df_teste_semana['Date'].dt.year

    # Calcular a variação percentual semanal para Price e Predicoes
    df_teste_semana['Variação Prevista (%)'] = df_teste_semana['predicoes'].pct_change() * 100

    # Remover NaN (primeira linha não terá variação por não ter valor anterior)
    # df_teste_semana = df_teste_semana.dropna()

    df_teste_semana.to_csv(f"{caminho_arquivos}\previsão semanal - {nome}.csv")

    return df_teste_semana


if ano > 2021:
    with ThreadPoolExecutor() as executor:
        # Execute as 4 chamadas de função em paralelo
        executor.submit(previsao, df_bitcoin, 100, 10, 20, 'Bitcoin')
        executor.submit(previsao, df_ethereum, 100, 10, 20, 'Ethereum')
        executor.submit(previsao, df_bnb, 100, 10, 20, 'BNB')
        executor.submit(previsao, df_solana, 100, 10, 20, 'Solana')
else:
    # Se o ano for menor ou igual a 2021, remove o arquivo da solana se tiver e gera um novo dos outros
    if os.path.exists(caminho_arquivos / 'previsão semanal - Solana.csv'):
        os.remove(caminho_arquivos / 'previsão semanal - Solana.csv')
    else: 
        pass
    
    with ThreadPoolExecutor() as executor:
        # Execute as 3 chamadas de função em paralelo
        executor.submit(previsao, df_bitcoin, 100, 10, 20, 'Bitcoin')
        executor.submit(previsao, df_ethereum, 100, 10, 20, 'Ethereum')
        executor.submit(previsao, df_bnb, 100, 10, 20, 'BNB')
    

[array([0.01131459, 0.01247838, 0.01335661, 0.01885766, 0.01098593,
       0.00543101, 0.00557648, 0.00622303, 0.00616376, 0.0064116 ,
       0.        , 0.00133081, 0.00212283, 0.00184266, 0.00192887,
       0.00225753, 0.00615837, 0.00466592, 0.00591592, 0.00579199,
       0.00724134, 0.00718746, 0.00733293, 0.00584049, 0.00587281,
       0.00701505, 0.00714436, 0.0071713 , 0.00695578, 0.00728983,
       0.00970361, 0.01061417, 0.01177795, 0.01226286, 0.01327579,
       0.01242989, 0.01289325, 0.01423484, 0.01455272, 0.01043098,
       0.01143313, 0.01200963, 0.01137386, 0.01131459, 0.01200963,
       0.01200425, 0.01328118, 0.01422406, 0.01438031, 0.01419712,
       0.01574345, 0.01797404, 0.01805486, 0.02082424, 0.02107208,
       0.0195958 , 0.02080808, 0.0217348 , 0.02176174, 0.0235613 ])]
[0.02532852732474502]
[array([0.01131459, 0.01247838, 0.01335661, 0.01885766, 0.01098593,
       0.00543101, 0.00557648, 0.00622303, 0.00616376, 0.0064116 ,
       0.        , 0.00133081, 0.002

c:\Users\yupopic\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
Epoch 1/20
Epoch 1/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - loss: 0.0127
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - loss: 0.0212
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 99ms/step - loss: 0.0030
Epoch 3/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0152
Epoch 2/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - loss: 0.0023
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - loss: 0.0025
Epoch 4/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 11s 100ms/step - loss: 0.0019
Epoch 3/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - loss: 0.0015
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - loss: 0.0022
Epoch 5/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - loss: 0.0016
67/73 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0021Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - loss: 0.0021
 85/104 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0016Epoch 6/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - loss: 0.0015
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - 